In [1]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error
import plotly.express as px
from sklearn.model_selection import train_test_split
import xgboost as xgb
from catboost import CatBoostRegressor
import tqdm
import lightgbm as lgb
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
def data_calculation_for_train(data,values, count_describe):
    count_describe=values
    # data = pd.DataFrame(pd.read_csv('/home/alex4/Desktop/AidisTech/hotel/data_by_days.csv'))
    df = pd.DataFrame(data.a[values - count_describe:values - 1].describe())
    # df.append(data.a[14])

    df2 = pd.DataFrame(
        [list(data.a[0:values])], columns=(range(values)))
    # df2=df2.pivot_table(index=df2.columns,columns='w')
    df2.index = ["a"]
    print(df2.index)
    df = df.pivot_table(values="a", columns=df.index)
    stat = pd.concat([df, df2], axis=1)
    print(stat)
    # print(data)
    for i in range(values + 1, len(data)):
        df = pd.DataFrame(data.a[i - count_describe:i - 1].describe())
        df = df.pivot_table(values="a", columns=df.index)
        df2 = pd.DataFrame([list(data.a[i - values:i])], columns=(range(values)))
        df2.index = ["a"]
        df = pd.concat([df, df2], axis=1)
        print(df)

        print(i)
        stat = stat.append(df)

    # df=pd.DataFrame(data.a[14 - 14:14].describe())
    stat = stat.drop(["count"], axis=1)
    print(stat)
    stat.to_csv('/home/alex4/Desktop/AidisTech/final_project/data_for_train.csv', index=False)


In [3]:
def exelread():
    data = pd.DataFrame(pd.read_excel(r'/home/alex4/Desktop/AidisTech/hotel/2019_01.xls', skiprows=8))
    print(data.info)
    data1 = pd.DataFrame(pd.read_excel('/home/alex4/Desktop/AidisTech/hotel/2019_02.xls', skiprows=8))
    data = data.append(data1)
    for i in range(2, 25):
        print(i)
        data1 = pd.DataFrame(
            pd.read_excel(f'/home/alex4/Desktop/AidisTech/hotel/Гостинница Северная, профиль мощности ({i}).xls',
                          skiprows=8))
        data = data.append(data1)
    print(data.info)

In [4]:
def my_plot_and_error(data,test_data):
    # data = pd.DataFrame(pd.read_csv('/home/alex4/Desktop/AidisTech/final_project/data_for_train.csv'))
    print(mean_absolute_percentage_error(data, test_data))
    plot(data)
    plot(test_data)
    # print(mean_absolute_percentage_error(Y, y_pred))
    # print("размер тестовой выборки= ", y_test.size)
    # print("размер обучающей выборки= ", y_train.size)


In [5]:
def test_models(data,times,values,score):
    res = {'forest': 0, 'GBM': 0, 'xgb': 0, 'cat': 0 ,'lgbm':0}
    data = pd.DataFrame(pd.read_csv('data_for_train.csv')).head(762-24)
    # data=data.sample(frac=1)
    Y = data[str(values)]
    data = data.drop([str(values)], axis=1)
    # score=0.05
    models=[]
    for i in tqdm.tqdm(range(times)):
        #     print()
        #     print(i)
        #     print()
        # regr = RandomForestRegressor(max_depth=10,n_estimators=100)
        X_train, X_test, y_train, y_test = train_test_split(data, Y, test_size=0.1,shuffle=True)
        regr = RandomForestRegressor(n_estimators=100, max_depth=5)
        # shuffle=True=>5%
        regr.fit(X_train, y_train)
        y_pred = regr.predict(X_test)
        error=  mean_absolute_percentage_error(y_test, y_pred)
        print("forest  ", error)
        res['forest'] += error
        if error<score:
            models.append(regr)
        regr = GradientBoostingRegressor(learning_rate=0.1, n_estimators=100, max_depth=5)
        # shuffle=True=>5%
        regr.fit(X_train, y_train)
        y_pred = regr.predict(X_test)
        error = mean_absolute_percentage_error(y_test, y_pred)
        print("GBM  ", error)
        res['GBM'] += error
        if error < score:
            models.append(regr)

        regr = xgb.XGBRegressor(max_depth=5, learning_rate=0.1)
        # shuffle=True=>5%
        regr.fit(X_train, y_train)
        y_pred = regr.predict(X_test)
        error = mean_absolute_percentage_error(y_test, y_pred)
        print("xgb  ", error)
        res['xgb'] += error
        if error <score:
            models.append(regr)

        regr = CatBoostRegressor(learning_rate=0.1, depth=5, logging_level='Silent')
        # shuffle=True=>5%
        regr.fit(X_train, y_train)
        y_pred = regr.predict(X_test)
        error = mean_absolute_percentage_error(y_test, y_pred)
        print("cat  ", error)
        res['cat'] += error
        if error <score:
            models.append(regr)
        regr = lgb.LGBMRegressor(max_depth=5, learning_rate=0.1)
        # shuffle=True=>5%
        regr.fit(X_train, y_train)
        y_pred = regr.predict(X_test)
        error = mean_absolute_percentage_error(y_test, y_pred)
        print("lgbm  ", error)
        res['lgbm'] += error
        if error < score:
            models.append(regr)
        # mean(cross_val_score(regr, X_train, y_train, cv=10))
    print("размер тестовой выборки= ", y_test.size)
    print("размер обучающей выборки= ", y_train.size)
    return models


In [6]:
def stacking (models,data):
    ans=list()
    for model in models:
        ans.append(model.predict(data))
    return sum(ans)/len(ans)

In [13]:
# def predict_n_times(data, values, count_describe, n, model):
def predict_n_times(data, values, count_describe, n, models):
    data=data.head(len(data)-n)
    result= pd.DataFrame(columns=["begin","a"])
    for i in range(len(data) - values, len(data) - values + n):
        tmp = pd.DataFrame(data.a[i: i + values].describe())
        tmp = tmp.drop(["count"], axis=0).T
        #         tmp
        tmp.index = ["w"]
        data_for_predict = pd.DataFrame(list(data.a[i:i + values])).T
        #         print(data_for_predict)
        data_for_predict.index = ["w"]
        data_for_predict = pd.concat([tmp, data_for_predict], axis=1)
        # data_for_predict
#         model.predict(data_for_predict)
#         data = data.append(pd.DataFrame(model.predict(data_for_predict), columns=["a"]), ignore_index=True)
        data = data.append(pd.DataFrame(stacking (models,data_for_predict), columns=["a"]), ignore_index=True)
        result=result.append(data.tail(1))

    return result


In [14]:
if __name__ == '__main__':
#     data = pd.DataFrame(pd.read_csv('/home/alex4/Desktop/AidisTech/hotel/data_by_days.csv'))
    data =pd.DataFrame(pd.read_csv('/home/alex4/Desktop/AidisTech/final_project/check_data.csv'))
    print("data was rеad")
    values=60
    n=24
#     data_calculation_for_train(data,values,1)

    models=test_models(1,1,values-1,score=0.10)
    y_pred=predict_n_times(data,values-1,n=n,models=models,count_describe=False)
    y_test=data.tail(n)
#     print(y_test.info)
#     print(y_pred.info)
    my_plot_and_error(y_test.a,y_pred.a)
#     s
    # X_train, X_test, y_train, y_test = train_test_split(data, Y, test_size=0.3)
    # model=
    print()

  0%|          | 0/3 [00:00<?, ?it/s]

data was rеad
forest   0.01549440072247573
GBM   0.011684124200385889
xgb   0.011836634393006803


 33%|███▎      | 1/3 [00:05<00:11,  5.69s/it]

cat   0.009071907372196486
lgbm   0.013058712082235998
forest   0.01530460239993211
GBM   0.010644396313520888
xgb   0.0094454561049275


 67%|██████▋   | 2/3 [00:11<00:05,  5.92s/it]

cat   0.008584885045241191
lgbm   0.01054065526146444
forest   0.013556142552572597
GBM   0.011620135004104937
xgb   0.010360801595917174


100%|██████████| 3/3 [00:17<00:00,  5.97s/it]

cat   0.009969877063132744
lgbm   0.011747913101128655
размер тестовой выборки=  69
размер обучающей выборки=  615


ValueError: feature_names mismatch: ['25%', '50%', '75%', 'max', 'mean', 'min', 'std', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58'] ['mean', 'std', 'min', '25%', '50%', '75%', 'max', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58']

In [16]:
# y_test.a
# y_pred.a
# =data.tail(n)
# my_plot_and_error(y_test.a,y_pred.a)
# plot(y_test.a)
# plot(y_pred.a)
# pylab.show()
# pd.DataFrame(pd.read_csv('/home/alex4/Desktop/AidisTech/final_project/check_data.csv'))
stacking (models,data_for_predict)

NameError: name 'data_for_predict' is not defined